In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sarcoma"
cohort = "GSE233860"

# Input paths
in_trait_dir = "../../input/GEO/Sarcoma"
in_cohort_dir = "../../input/GEO/Sarcoma/GSE233860"

# Output paths
out_data_file = "../../output/preprocess/Sarcoma/GSE233860.csv"
out_gene_data_file = "../../output/preprocess/Sarcoma/gene_data/GSE233860.csv"
out_clinical_data_file = "../../output/preprocess/Sarcoma/clinical_data/GSE233860.csv"
json_path = "../../output/preprocess/Sarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE233860_family.soft.gz', 'GSE233860_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Sarcoma/GSE233860/GSE233860_family.soft.gz
Matrix file: ../../input/GEO/Sarcoma/GSE233860/GSE233860_series_matrix.txt.gz
Background Information:
!Series_title	"Paired tumour biopsy gene expression data from patients with solid tumours, before and after combination treatment."
!Series_summary	"Gene expression quantification of PanCancer IO genes from paired tumour biopsies from 24 patients with pan-cancer solid tumours, before and after treatment with MTL-CEBPA and pembrolizumab."
!Series_overall_design	"Normalised, transformed gene counts of nCounter Nanostring PanCancer IO panel (770 genes) of paired tumour biopsies from 24 patients with solid tumours, before treatment (Screening) and 6 weeks after treatment (C2D16) with MTL-CEBPA and pembrolizumab."
Sample Characteristics Dictionary:
{0: ['outcome: SD', 'outcome: PD', 'outcome: PR']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze dataset characteristics

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# It mentions "Gene expression quantification of PanCancer IO genes" and "Normalised, transformed gene counts"
is_gene_available = True  

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Sarcoma)
# Looking at the sample characteristics, there's no direct mention of sarcoma
# The dataset is about "pan-cancer solid tumours" which may include sarcoma, but no specific indication
trait_row = None  # No specific indication that this dataset contains sarcoma data

# For age
# No age information is available in the sample characteristics
age_row = None

# For gender
# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since we don't have trait, age, or gender data available, we'll define simple functions 
# that would handle the conversion if the data were available

def convert_trait(value):
    if value is None:
        return None
    # If we had trait data, we would extract the value after colon and convert to binary
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Return 1 if it's a sarcoma case, 0 otherwise
    return 1 if 'sarcoma' in value.lower() else 0

def convert_age(value):
    if value is None:
        return None
    # If we had age data, we would extract the numeric value
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if value is None:
        return None
    # If we had gender data, we would convert to binary
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save the cohort information using the provided function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 770
First 20 gene/probe identifiers:
Index(['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1',
       'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B',
       'API5', 'APLNR', 'APOE', 'APOL6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# Based on biomedical knowledge, these appear to be standard human gene symbols (like A2M, ABCF1, etc.)
# These are recognized official gene symbols, not probe IDs or other identifiers that would need mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
print(f"Sample IDs in gene data: {gene_data.columns[:5].tolist()}...")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since we determined in Step 2 that trait_row is None (no sarcoma data available),
# we cannot properly link clinical and genetic data for our target trait
print("Based on earlier analysis, this dataset does not contain sarcoma trait information.")

# Create a minimal dataframe with the trait column (all zeros as placeholder)
# This allows our functions to run properly even though the data is not usable for analysis
linked_data = gene_data.T.copy()
linked_data[trait] = 0  # Add placeholder trait column

# 3. Handle missing values in the linked data
# Even though we know the data isn't usable, we run this to satisfy function requirements
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 4. Check if the trait and demographic features are biased
# Using a placeholder dataframe with trait column so the function runs properly
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# Create a minimal clinical dataframe to save
clinical_df = pd.DataFrame(index=linked_data.index)
clinical_df[trait] = 0  # Add placeholder trait column
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data (placeholder) saved to {out_clinical_data_file}")

# 5. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from pan-cancer solid tumors before and after treatment, but does not contain specific sarcoma trait information. Placeholder trait values were added for technical validation only."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # We determined in step 2 that trait data is not available
    is_biased=False,  # Providing a value as required, though it's not meaningful since trait data is absent
    df=unbiased_linked_data,
    note=note
)

# We already know the data is not usable for our purposes, but we'll check the flag anyway
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Shape of gene data after normalization: (762, 48)
First few gene symbols: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1']
Sample IDs in gene data: ['GSM7438145', 'GSM7438146', 'GSM7438147', 'GSM7438148', 'GSM7438149']...
Normalized gene data saved to ../../output/preprocess/Sarcoma/gene_data/GSE233860.csv
Based on earlier analysis, this dataset does not contain sarcoma trait information.


Shape of linked data after handling missing values: (48, 763)


Quartiles for 'Sarcoma':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0
Max: 0
The distribution of the feature 'Sarcoma' in this dataset is severely biased.

Clinical data (placeholder) saved to ../../output/preprocess/Sarcoma/clinical_data/GSE233860.csv
Dataset validation failed. Final linked data not saved.
